# Tratamento de Lista Geral de Leads
## Objetivo: Esse pipeline tem como objetivo separar a lista de Leads disponível para disparos

### Campanha: lcto-ofan-jan26
### Conversion types: 1


In [8]:
# Importações básicas
import pandas as pd
import numpy as np
import sys
from pathlib import Path
import os
sys.path.insert(0, os.path.join(os.path.dirname(os.getcwd()), 'src'))

# Adiciona src ao path
sys.path.append('../src')

#

# Utilitários de dados
from data_utils import (
    load_raw_data,
    save_processed_data,
    remove_duplicates,
    handle_missing_values,
    detect_outliers,
    normalize_column,
    process_phone_string,
    process_phone_number,
    clean_and_lower_column,
    flatten_list_to_df,
    remove_buyers_from_dataframe
)

CRONOGRAMA_SUBDOMAIN = 'cronogramadosfluentes-xwamel'

# Utilitários SQL
from sql_utils import DatabaseConnection as Dbc, load_query_from_file

# Utilitários de visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Utilitários de API
from api_utils import (
    make_request,
    get_json,
    post_json,
    paginated_request,
    response_to_dataframe
)

# utilitários hotmart
from hotmart_utils import Hotmart

# utilitários tmb
from tmb_utils import TMB   

# Configurações pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Load Database Driver
db = Dbc()

# Inicializar API Hotmart
hotmart = Hotmart()

# Inicializar API TMB
tmb = TMB()

print('✓ Importações concluídas com sucesso!')

✓ Importações concluídas com sucesso!


## Load Leads Dataframe

In [9]:
# Campaign Variables

CAMPAIGN_ID = 'lcto-ofan-jan26'
MAIN_CONVERSION_TYPE_ID = 1

# Load Leads Dataframe
df = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": CAMPAIGN_ID, "conversion_type_id": MAIN_CONVERSION_TYPE_ID})
df_2 = db.execute_query_from_file("select_complete_lead_data_from_campaign", params={"campaign_id": CAMPAIGN_ID, "conversion_type_id": 2})

# Lead Offers and Sales to Eval
df_offers = db.execute_query_from_file("select_all_offers")

df_sales = flatten_list_to_df(hotmart.get_sales_history(
    start_date='2025-12-20', end_date='2025-12-31'
)) 

merged_df_sales = pd.merge(df_sales, df_offers, left_on='purchase_offer_code', right_on='offer_id', how='left')

## Relatório de Qualificação e Origens


In [10]:
print('Entradas nos Grupos: ')
print('Saidas dos Grupos: ')
print(f'Respostas à pesquisa: {df_2["lead_id"].nunique()}')
print('')

agg = (
    df.assign(
        lead_id_renda_true=df["lead_id"].where(df["renda_qualificada"] == True),
        lead_id_qual_true=df["lead_id"].where(df["qualificado_geral"] == True),
    )
    .groupby("origem", dropna=False)
    .agg(
        lead_id_nunique=("lead_id", "nunique"),
        renda_true_nunique=("lead_id_renda_true", "nunique"),
        qualificado_true_nunique=("lead_id_qual_true", "nunique"),
    )
    .reset_index()
)

# % dentro de cada origem
agg["pct_renda_true"] = (agg["renda_true_nunique"] / agg["lead_id_nunique"]).mul(100)
agg["pct_qualificado_true"] = (agg["qualificado_true_nunique"] / agg["lead_id_nunique"]).mul(100)

# linha TOTAL (percentuais recalculados no total)
total_leads = agg["lead_id_nunique"].sum()
total_renda = agg["renda_true_nunique"].sum()
total_qual = agg["qualificado_true_nunique"].sum()

agg_total = pd.DataFrame([{
    "origem": "TOTAL",
    "lead_id_nunique": total_leads,
    "renda_true_nunique": total_renda,
    "qualificado_true_nunique": total_qual,
    "pct_renda_true": (total_renda / total_leads) * 100 if total_leads else 0.0,
    "pct_qualificado_true": (total_qual / total_leads) * 100 if total_leads else 0.0,
}])

agg = pd.concat([agg, agg_total], ignore_index=True)

agg

Entradas nos Grupos: 
Saidas dos Grupos: 
Respostas à pesquisa: 13692



,origem,lead_id_nunique,renda_true_nunique,qualificado_true_nunique,pct_renda_true,pct_qualificado_true
0,Orgânico,8858,3353,1998,37.852788,22.555882
1,Pago,12798,3714,2176,29.020159,17.002657
2,TOTAL,21656,7067,4174,32.632989,19.274104


## Filter Buyers From Dataframe

In [11]:
## Filter Buyers from Dataframe
df = remove_buyers_from_dataframe(df)

✓ Removidos 411 compradores de 21,656 registros (1.9% filtrado)


In [12]:
# garante datetime
df["last_conversion_date"] = pd.to_datetime(df["last_conversion_date"], format="%Y-%m-%d", errors="coerce")

# filtra apenas os leads do dia 31/12/2025 até ontem
inicio = pd.Timestamp("2025-12-31")
ontem = pd.Timestamp.now().normalize() - pd.Timedelta(days=1)
df_ontem = df[(df["last_conversion_date"] >= inicio) & (df["last_conversion_date"] <= ontem)].copy()

In [13]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(merged_df_sales):
    # Filter rows based on column: 'purchase_recurrency_number'
    merged_df_sales = merged_df_sales[(merged_df_sales['purchase_recurrency_number'] == 1) | (merged_df_sales['purchase_recurrency_number'].isna())]
    return merged_df_sales

merged_df_sales_clean = clean_data(merged_df_sales.copy())
merged_df_sales_clean.head()


merged_leads = pd.merge(merged_df_sales_clean, df, left_on='buyer_email', right_on='email', how='left')

# Format 3C+ Base

In [14]:
df_3c = df.copy()

# Define type como nacional/internacional
df_3c['type'] = df_3c['phone'].astype(str).apply(lambda x: 'nacional' if x.startswith('+55') else 'internacional')
df_3c = df_3c[df_3c['type'] == 'nacional']

# Cria a coluna br_phone removendo o prefixo +55 do phone
df_3c['br_phone'] = df_3c['phone'].astype(str).str.replace(r'^\+55', '', regex=True)

# DDD: primeiros 2 caracteres de br_phone
df_3c['DDD'] = df_3c['br_phone'].str[:2]

# br_simple_phone: caracteres restantes após o DDD
df_3c['br_simple_phone'] = df_3c['br_phone'].str[2:]

# Slices (conferindo possíveis nomes das colunas e True/False lowercase)
qualificados = df_3c[
    (df_3c['renda_qualificada'] == True) | (df_3c['qualificado_geral'] == True)
]

nao_qualificados = df_3c[
    (df_3c['renda_qualificada'] == False) & (df_3c['qualificado_geral'] == False)
]
